In [ ]:
print(spark)

* Next we are going to create an external table in the delta format referencing the location where we had created the external table in the Synapse serverless pool. This is an external table, so the table is not managed by spark and its corresponding warehouse. 

In [ ]:
%%sql
CREATE TABLE temperatures_ext_delta ( 
    latitude float, 
    longitude float, 
    time string,
	temperature_2m float
) USING DELTA
LOCATION "abfss://datalake@venkydatalake1001.dfs.core.windows.net/temperatures_delta/"

* Note that the external table was created inside the notebook. As soon as this executes, the folder we had referenced before in the Synapse serverless pool will get the _delta_log folder and it magically gets transformed into a delta table that we can read from inside the serverless pool!
* Next we will start creating external tables for the paths we referenced before and merge the data back to the delta table.

In [ ]:
%%sql
CREATE EXTERNAL TABLE temperatures_2018 
USING PARQUET 
LOCATION "abfss://datalake@venkydatalake1001.dfs.core.windows.net/temperatures/AirQualityIndexWithTemperatures_5/"

In [ ]:
%%sql

CREATE EXTERNAL TABLE temperatures_2019
USING PARQUET 
LOCATION "abfss://datalake@venkydatalake1001.dfs.core.windows.net/temperatures/AirQualityIndexWithTemperatures_4/";

CREATE EXTERNAL TABLE temperatures_2020 
USING PARQUET 
LOCATION "abfss://datalake@venkydatalake1001.dfs.core.windows.net/temperatures/AirQualityIndexWithTemperatures_3/";

CREATE EXTERNAL TABLE temperatures_2021 
USING PARQUET 
LOCATION "abfss://datalake@venkydatalake1001.dfs.core.windows.net/temperatures/AirQualityIndexWithTemperatures_2/";

CREATE EXTERNAL TABLE temperatures_2022 
USING PARQUET 
LOCATION "abfss://datalake@venkydatalake1001.dfs.core.windows.net/temperatures/AirQualityIndexWithTemperatures_1/";

CREATE EXTERNAL TABLE temperatures_2023
USING PARQUET 
LOCATION "abfss://datalake@venkydatalake1001.dfs.core.windows.net/temperatures/AirQualityIndexWithTemperatures_0/"

* As we can see there are multiple lines output. Since all these are external table definitions, there is nothing much to show.

In [ ]:
%%sql
SHOW TABLES

* As we can see all the tables we have defined are visible inside the spark layer. Let us merge all the tables to the main delta table.

In [ ]:
%%sql
MERGE INTO temperatures_ext_delta AS TARGET
USING temperatures_2018 AS SOURCE
ON TARGET.latitude = SOURCE.latitude AND
TARGET.longitude = SOURCE.longitude AND 
TARGET.`time` = SOURCE.`time` 
WHEN MATCHED THEN 
UPDATE SET 
    TARGET.temperature_2m = SOURCE.temperature_2m  
WHEN NOT MATCHED THEN
INSERT 
    ( TARGET.latitude, TARGET.longitude, TARGET.`time`, TARGET.temperature_2m )
VALUES 
    ( SOURCE.latitude, SOURCE.longitude, SOURCE.`time`, SOURCE.temperature_2m );

MERGE INTO temperatures_ext_delta AS TARGET
USING temperatures_2019 AS SOURCE
ON TARGET.latitude = SOURCE.latitude AND
TARGET.longitude = SOURCE.longitude AND 
TARGET.`time` = SOURCE.`time` 
WHEN MATCHED THEN 
UPDATE SET 
    TARGET.temperature_2m = SOURCE.temperature_2m  
WHEN NOT MATCHED THEN
INSERT 
    ( TARGET.latitude, TARGET.longitude, TARGET.`time`, TARGET.temperature_2m )
VALUES 
    ( SOURCE.latitude, SOURCE.longitude, SOURCE.`time`, SOURCE.temperature_2m );

MERGE INTO temperatures_ext_delta AS TARGET
USING temperatures_2020 AS SOURCE
ON TARGET.latitude = SOURCE.latitude AND
TARGET.longitude = SOURCE.longitude AND 
TARGET.`time` = SOURCE.`time` 
WHEN MATCHED THEN 
UPDATE SET 
    TARGET.temperature_2m = SOURCE.temperature_2m  
WHEN NOT MATCHED THEN
INSERT 
    ( TARGET.latitude, TARGET.longitude, TARGET.`time`, TARGET.temperature_2m )
VALUES 
    ( SOURCE.latitude, SOURCE.longitude, SOURCE.`time`, SOURCE.temperature_2m );

MERGE INTO temperatures_ext_delta AS TARGET
USING temperatures_2021 AS SOURCE
ON TARGET.latitude = SOURCE.latitude AND
TARGET.longitude = SOURCE.longitude AND 
TARGET.`time` = SOURCE.`time` 
WHEN MATCHED THEN 
UPDATE SET 
    TARGET.temperature_2m = SOURCE.temperature_2m  
WHEN NOT MATCHED THEN
INSERT 
    ( TARGET.latitude, TARGET.longitude, TARGET.`time`, TARGET.temperature_2m )
VALUES 
    ( SOURCE.latitude, SOURCE.longitude, SOURCE.`time`, SOURCE.temperature_2m );

MERGE INTO temperatures_ext_delta AS TARGET
USING temperatures_2022 AS SOURCE
ON TARGET.latitude = SOURCE.latitude AND
TARGET.longitude = SOURCE.longitude AND 
TARGET.`time` = SOURCE.`time` 
WHEN MATCHED THEN 
UPDATE SET 
    TARGET.temperature_2m = SOURCE.temperature_2m  
WHEN NOT MATCHED THEN
INSERT 
    ( TARGET.latitude, TARGET.longitude, TARGET.`time`, TARGET.temperature_2m )
VALUES 
    ( SOURCE.latitude, SOURCE.longitude, SOURCE.`time`, SOURCE.temperature_2m );

MERGE INTO temperatures_ext_delta AS TARGET
USING temperatures_2023 AS SOURCE
ON TARGET.latitude = SOURCE.latitude AND
TARGET.longitude = SOURCE.longitude AND 
TARGET.`time` = SOURCE.`time` 
WHEN MATCHED THEN 
UPDATE SET 
    TARGET.temperature_2m = SOURCE.temperature_2m  
WHEN NOT MATCHED THEN
INSERT 
    ( TARGET.latitude, TARGET.longitude, TARGET.`time`, TARGET.temperature_2m )
VALUES 
    ( SOURCE.latitude, SOURCE.longitude, SOURCE.`time`, SOURCE.temperature_2m );







* As we can see all the merges completed pretty quick and the data was merged into the delta table. We can query the data now to see the rows. 

In [ ]:
%%sql
SELECT COUNT(*) FROM temperatures_ext_delta